# First Deliverable - Extract Cities and Countries

import of used modules

In [1]:
import geonamescache
import re
import unidecode
import pandas as pd

City and country data is held within dictionarys and can be summoned thus

In [2]:
gc = geonamescache.GeonamesCache()
cities = gc.get_cities()
countries = gc.get_countries()

***UPDATE RevB***
I have realised that there are a few things that need correcting regarding this workflow.
1) The unidecode can be taken out of the loop and done once on the        cityRegex string as a whole.
2) Adding the cities directly into the regex string does not account      for partial matches 'San' instead of 'San Diego'.
   To acount for this the best thing would be to collect the cities      into a list before adding to the regex string.  
   This list can then be sorted by length of name.  'San Diego' would    then be found before 'San' when re.search() is carried out.

In [3]:
cityList=[]

for city in cities:
    cityList.append(cities[city]['name'])

# once all cities extracted, they can be sorted by name length

sortedCities = sorted(cityList, key=len, reverse=True)
#print(sortedCities)

The data in the list can now be added into one long string which will be our regex matching pattern and then all the unicode and non alpha characters need to be removed

In [4]:
cityString = ""
for city in sortedCities:
    cityString = cityString + city + "|"

# removal of any diacritic chars
cityString = unidecode.unidecode(cityString)

# Also need to remove the "()" chars which are embedded into the geonamescache library 
# e.g Kempten (Allgau)
cityString = re.sub(r"\(|\)", "", cityString)

# Affix with the regex group 
cityRegex = r"(?P<cityName>(\b(" + cityString[:-1] + r")\b))"

#print(cityRegex)

To make the matching process much quicker, the regex patter is compiled...

In [5]:
compiledCity = re.compile(cityRegex) #, flags=re.IGNORECASE)
compiledCityI = re.compile(cityRegex, flags=re.IGNORECASE)

The same process needs to be carried out for the country dictionary

In [6]:
countryRegex = r"\b("
for country in countries:
    countryClense = re.sub(r"\(|\)", "", countries[country]['name'])
    countryRegex = countryRegex + unidecode.unidecode(countryClense) + "|"

countryRegex = countryRegex[:-1] + r")\b"
countryRegexCompiled = re.compile(countryRegex, flags=re.IGNORECASE)


In [7]:
countryList = []

for country in countries:
    countryList.append(countries[country]['name'])

sortedCountries = sorted(countryList, key=len, reverse=True)
#print(sortedCountries)   

In [8]:
countryString = ""
for country in countryList:
    countryString = countryString + country + "|"
    
countryString = unidecode.unidecode(countryString)
countryString = re.sub(r"\(|\)", "", countryString)
countryRegex = r"(?P<countryName>(\b(" + countryString[:-1] + r")\b))"

#print(countryRegex)
compiledCountry = re.compile(countryRegex) #, flags=re.IGNORECASE)

Now we have regex pattern, we need to match this against all the headlines.  A record of matches or non matches is kept in a new dictionary declared before the matching process

In [9]:
dictionary = { "headline": [], "city": [], "country": [] }

I have used the regex method 'findall()' as this will take into account multiple name matches per headline.  This returns a list of matched results and an empty list if no matches.

The process below takes each line and pattern matches against both city and country.

If the match is 1 then the city name is added to our dictionary.

If the pattern matches 2 names in the same headline the longest string found is returned so that partial matches like 'San' or 'Of' don't get taken into account. 

In [10]:
with open(r'data/headlines.txt', 'r') as data:
    for line in data:
        dictionary['headline'].append(line.rstrip('\n'))
        countryMatch = re.search(compiledCountry, line)
        if countryMatch:
            dictionary['country'].append(countryMatch.group('countryName'))
        else:
            dictionary['country'].append("NaN")

        cityMatch  = re.search(compiledCity, line)
        if cityMatch:
            dictionary['city'].append(cityMatch.group('cityName'))
        else:
            cityMatch = re.findall(compiledCityI, line)
            if len(cityMatch) >= 1:
                matches = [name[0] for name in cityMatch]
                dictionary['city'].append(max(matches))
                # print(max(matches))
            else:
                dictionary['city'].append("NaN")

Rio De Janeiro
Mcallen
Mclean
hit
of
come
of
of
of
of
of


In [11]:
#print(dictionary)

Our dictionary, now filled with matches or non matches can be converted to a pandas dataframe...

In [12]:
df = pd.DataFrame(dictionary)

In [13]:
df.style

,headline,city,country
0,Zika Outbreak Hits Miami,Miami,NaN
1,Could Zika Reach New York City?,New York City,NaN
2,First Case of Zika in Miami Beach,Miami Beach,NaN
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil
4,Dallas man comes down with case of Zika,Dallas,NaN
5,Trinidad confirms first Zika case,Trinidad,NaN
6,Zika Concerns are Spreading in Houston,Houston,NaN
7,Geneve Scientists Battle to Find Cure,Geneve,NaN
8,The CDC in Atlanta is Growing Worried,Atlanta,NaN
9,Zika Infested Monkeys in Sao Paulo,Sao Paulo,NaN


In [14]:
df.to_csv('data/df1.csv', sep=';')